# Yolov8n model

In [42]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt") # pretrained COCO model laden

In [43]:
model.model # Conv = Convolutional kernel, bn = batch normalization, act = activation function

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    

# Image Controle
Voordat we de images gaat splitsen in een training en validatie set moeten we controleren of de images geopend kunnen worden vanuit Python. Dit voorkomt dat we in de volgende stappen tegen fouten aanlopen

In [65]:
import cv2
raw_images_folder = os.getcwd()+'/raw images/'
for element in os.listdir(raw_images_folder):
    image = cv2.imread(raw_images_folder+element)
    if image is None:
        print(f"Fout: Kan afbeelding {element} niet laden. Controleer het bestandspad.")
print("Alle afbeeldingen kunnen zonder problemen ingeladen worden")

Alle afbeeldingen kunnen zonder problemen ingeladen worden


# Splits Dataset in Training set and Validation set

Nu gaan we de augmented images opsplitsen in training data en validation data. 80% van de augmented data gaan we gebruiken voor het finetunen van het YOLOv8n model en 20% gebruiken we voor de validatie. In de volgende stap gaan we de training set augmenteren.

In [66]:
import os 
import shutil
from sklearn.model_selection import train_test_split
import random

# Ophalen van alla images
raw_images_path = os.getcwd() + '/raw images'
raw_images = os.listdir(raw_images_path) 
random.shuffle(raw_images)

# Instellen van Paden
train_images_path = os.getcwd() + "/images/train"
val_images_path = os.getcwd() + "/images/val"

# Splitsen van dataset in train en validation dataset
train_images, val_images = train_test_split(raw_images, test_size=0.2)

# Nu gaan we de images naar de juiste plaats verplaatsen
for img in train_images:
    shutil.copy(os.path.join(raw_images_path, img), train_images_path)
for img in val_images:
    shutil.copy(os.path.join(raw_images_path, img), val_images_path)

print(f"De dataset bestaat uit {len(raw_images)} images")
print(f"De training set bestaat uit {0.8*len(raw_images)} images")
print(f"De validatie set bestaat uit {0.2*len(raw_images)} images")

De dataset bestaat uit 450 images
De training set bestaat uit 360.0 images
De validatie set bestaat uit 90.0 images


# Data Augmentation

In deze stap ga ik data augmentatie toepassen op de training set. Dit is een veelgebruikte techniek om je dataset te vergroten. Het voert allerlei image transformaties toe op je input images en slaat ze vervolgens op. Dit is een goede en snelle manier om je training set te verrijken en verhelpt deels het probleem van een te kleine training set. Zorg er wel voor dat de transformaties realistisch zijn. Na inspectie merkte ik op dat er van sommige klassen minder dan 15 images zaten in de validatie set. Vandaar dat ik een paar images verplaatst heb vanuit de training set naar de validatie set zodat elke klasse minstens 15 validatie images heeft. Vergeet niet om de verplaatste images volledig te verwijderen uit de training set om kruisbestuiving tussen de training en validatie set te voorkomen.

In [67]:
# Laten we eerst Data Augmentatie toepassen
import albumentations as alb
import os
import cv2

# Aantal images in de training set en validatie set voor augmentatie 
train_images_folder = os.getcwd() + "/images/train"
train_images = os.listdir(train_images_path)
val_images_folder = os.getcwd() + "/images/val"
val_images = os.listdir(val_images_path)
print(f"De training set bestaat uit {len(train_images)} images")
print(f"De validatie set bestaat uit {len(val_images)} images")

# Bepaal nu alle transformaties die je wilt toepassen op de foto's 
transform = alb.Compose([
    alb.RandomRotate90(p=0.5),
    alb.HorizontalFlip(p=0.5),
    alb.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    alb.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    alb.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.3),
    alb.Blur(blur_limit=3, p=0.2),
    alb.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
    alb.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=0.2)
])

# Laten we de transformaties toepassen op onze images
num_augmented_images = 2 # aantal augmentaties per afbeelding

for image_name in train_images:
    train_image_path = os.path.join(train_images_folder, image_name)
    image = cv2.imread(train_image_path)

    for i in range(num_augmented_images):
        augmented = transform(image=image)
        augmented_image = augmented["image"]
        output_path = os.path.join(train_images_folder, f"aug_{i}_{image_name}")
        cv2.imwrite(output_path, augmented_image)

train_images = os.listdir(train_images_path)
val_images = os.listdir(val_images_path)
print(f"De training set bestaat uit {len(train_images)} images")
print(f"De validatie set bestaat uit {len(val_images)} images")

De training set bestaat uit 353 images
De validatie set bestaat uit 97 images
Afbeelding bank10.jpeg succesvol geladen
Afbeelding bank11.jpeg succesvol geladen
Afbeelding bank12.jpeg succesvol geladen
Afbeelding bank13.jpeg succesvol geladen
Afbeelding bank14.jpeg succesvol geladen
Afbeelding bank15.jpeg succesvol geladen
Afbeelding bank16.jpeg succesvol geladen
Afbeelding bank20.jpeg succesvol geladen
Afbeelding bank21.jpeg succesvol geladen
Afbeelding bank22.jpeg succesvol geladen
Afbeelding bank23.jpeg succesvol geladen
Afbeelding bank24.jpeg succesvol geladen
Afbeelding bank25.jpeg succesvol geladen
Afbeelding bank26.jpeg succesvol geladen
Afbeelding bank27.jpeg succesvol geladen
Afbeelding bank28.jpeg succesvol geladen
Afbeelding bank29.jpeg succesvol geladen
Afbeelding bank30.jpeg succesvol geladen
Afbeelding bank31.jpeg succesvol geladen
Afbeelding bank32.jpeg succesvol geladen
Afbeelding bank33.jpeg succesvol geladen
Afbeelding bank34.jpeg succesvol geladen
Afbeelding bank38.jp

# Train het Model
In deze module gaan we het pretrained YOLOv8n model trainen op onze custom dataset. Ik wil dat het model na het trainen in staat is om de 80 classes van de COCO dataset te herkennen. Daarnaast wil ik het finetunen met mijn dataset zodat hij ook mijn custom classes kan herkennen. Als test heb ik maar 1 custom class gebruikt in mijn custom dataset. Als het werkt dan zal ik er meerdere classes aan toevoegen. Dus voor nu herkent hij 81 classes

In [31]:
# Dit is het totale aantal layers
num_layers = len(list(model.model.model))
print(f"Totale aantal lagen: {num_layers}")


# Nu gaan we het model trainen
model.train(
    data = "custom_dataset.yaml",
    epochs = 50,
    batch = 8,
    patience = 10,
    freeze = num_layers-1,
    val = True
)

Totale aantal lagen: 23
Ultralytics 8.3.36  Python-3.9.20 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=custom_dataset.yaml, epochs=50, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=22, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

train: Scanning C:\Users\OussamaAkhiyat\Documents\Python Codes\Omni\labels\train.cache... 120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\OussamaAkhiyat\Documents\Python Codes\Omni\labels\val.cache... 30 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30/30 [00:00<?, ?it/s]

Plotting labels to runs\detect\train\labels.jpg... 


...

                   all         30         30      0.808      0.985      0.962      0.627
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 39, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



49 epochs completed in 0.664 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.5MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.5MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.36  Python-3.9.20 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
Model summary (fused): 168 layers, 3,160,775 parameters, 0 gradients, 8.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.23s/it]


                   all         30         30      0.981      0.967      0.994      0.736
               Oussama         30         30      0.981      0.967      0.994      0.736
Speed: 2.7ms preprocess, 155.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([80])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000021DD3CBFB20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048

# Model Testen

In [41]:
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt

best_weights_path = os.getcwd() + '/runs/detect/train/weights/best.pt'
test_image_path = os.getcwd() + '/cat.JPG'

trained_model = YOLO("runs/detect/train/weights/best.pt") 
results = model.predict(test_image_path, save=True)

result_image = results[0].plot()
plt.imshow(result_image)
plt.axis('off')
plt.show()


image 1/1 C:\Users\OussamaAkhiyat\Documents\Python Codes\Omni\cat.JPG: 640x640 2 Oussamas, 331.8ms
Speed: 9.9ms preprocess, 331.8ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\train4


<Figure size 640x480 with 1 Axes>